In [ ]:
%env OPENAI_API_KEY =
%env QDRANT_URL = 
%env QDRANT_API_KEY = 
%env LANGCHAIN_API_KEY=
LANGCHAIN_TRACING_V2="true"

In [3]:
from langchain_openai import ChatOpenAI
from typing import Sequence
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

In [4]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [6]:
model = ChatOpenAI(model="gpt-3.5-turbo")
parser = StrOutputParser()

In [7]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [8]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Dime todos los planetas del sistema solar."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Los planetas del sistema solar son:
1. Mercurio
2. Venus
3. Tierra
4. Marte
5. Júpiter
6. Saturno
7. Urano
8. Neptuno
Además, Plutón fue considerado un planeta en el pasado, pero ahora se le clasifica como un "planeta enano".


In [22]:
query = "Dime hasta qué mes de que año tienes información"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Mi información se encuentra actualizada hasta septiembre de 2021. Si necesitas información más reciente o tienes alguna otra pregunta, estaré encantado de ayudarte en lo que pueda. ¡No dudes en consultarme!
